In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import re
import datetime 
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestRegressor


def process_and_merge(fgIndex):
    
    # pd.set_option("display.max_rows", None, "display.max_columns", None)
    # print(fgIndex)
    
    fgIndex['date'] = pd.to_datetime(fgIndex['date'])
    fgIndex['date'] = fgIndex['date'].dt.date
    fgIndex = fgIndex.sort_values(by='date')
    fgIndex = fgIndex.reset_index(level = None, drop = True)
    
    
    # print(type(fgIndex['date'][12]))
    
    bit_info = pd.read_csv('coin_Bitcoin.csv', parse_dates=['Date'])    
    bit_info['date'] = pd.to_datetime(bit_info['Date'])
    mask = (bit_info['date'] > '2018-02-02')
    bit_info = bit_info.loc[mask]
    bit_info['date'] = bit_info['date'].dt.date
    bit_info = bit_info.drop(['Date'], axis = 1)
    bit_info = bit_info.reset_index(level = None, drop = True)
    print(type(bit_info['date'][3]))
    
    
    joined_data = bit_info.merge(fgIndex, on = "date", how='inner')
    # print(bit_info['date'][3] == fgIndex['date'][12])
    # print(joined_data)
    
    
    
   # bit_info = bit_info.drop(['SNo', 'Name', 'Symbol'], axis=1)
   # bit_info['Date'] = pd.to_datetime(bit_info['Date']).dt.date
   # bit_info = bit_info.groupby('Date').sum()

    #merging = daily_full.merge(bit_info, left_index=True, right_index=True)
    
    return joined_data


def run_model(df):

    X = df.drop(['High', 'Low', 'Close'], axis=1)
    y = df.Close
    
    # Test train split into the first 80% as our train, and last 20% for test
    X_train = X.iloc[0:int(0.8*X.shape[0])]
    X_valid = X.iloc[int(0.8*X.shape[0]):]
    y_train = y.iloc[0:int(0.8*y.shape[0])]
    y_valid = y.iloc[int(0.8*y.shape[0]):]

    estimators = 300
    depth = 7
    split = 50

    model = make_pipeline(
            StandardScaler(),
            RandomForestRegressor(n_estimators=estimators,
            max_depth=depth , min_samples_split=split, min_samples_leaf=15)
        )

    model.fit(X_train, y_train)
    # ONLY INCLUDE SCORE IN THE FINAL SUBMISSION
    print(model.score(X_valid, y_valid))
    print(model.score(X_train, y_train))
    
def main():
    
    df = data_extract()
    df = process_and_merge(df)
    #run_model(df)
    
    
if __name__== '__main__':
    main()

<class 'datetime.date'>


In [49]:
def data_extract():
    response_API = requests.get('https://api.alternative.me/fng/?limit=0&format=csv')
    data = response_API

    dataset = data.text.splitlines()
    # print(dataset)
    df = pd.DataFrame(dataset)
    df = df.drop(df.index[0:4])
    df.reset_index()
    df = df.drop(df.index[1399:])
    # print(df)

    tmpDF = pd.DataFrame(columns=['date','index', 'fear'])
    tmpDF[['date','index','fear']] = df[0].str.split(',',expand=True)
    
    return tmpDF
